In [22]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tf.reset_default_graph()

In [23]:
import numpy as np

In [27]:
input_train = [[[0],[0],[1],[2],[3]],[[2],[3],[4],[5],[6]],[[3],[4],[5],[6],[7]],[[4],[5],[6],[7],[8]],[[5],[6],[7],[8],[9]]]
output_train_val = [4,7,8,9,10]

output_train = []
for t in range(0,5):
    output_train_k = [0] * 11
    output_train_k[output_train_val[t]] = 1
    output_train.append(output_train_k)

input_np = np.array(input_train)
output_np = np.array(output_train)

input_train = input_np.reshape(input_np.shape[0],5,input_np.shape[2])
output_train = output_np.reshape(output_np.shape[0],11,)

print("Input shape:",str(input_train.shape))

print("\nInput Data:\n",str(input_train))

print("\nOutput shape:",str(output_train.shape))

print("\nOutput Data:\n",str(output_train))


Input shape: (5, 5, 1)

Input Data:
 [[[0]
  [0]
  [1]
  [2]
  [3]]

 [[2]
  [3]
  [4]
  [5]
  [6]]

 [[3]
  [4]
  [5]
  [6]
  [7]]

 [[4]
  [5]
  [6]
  [7]
  [8]]

 [[5]
  [6]
  [7]
  [8]
  [9]]]

Output shape: (5, 11)

Output Data:
 [[0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1]]


In [29]:
# Training Parameters
learning_rate = 0.001
training_steps = 1000
batch_size = 5
display_step = 200

# Network Parameters
num_input = 1 
timesteps = 5 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 11 #  total classes (0-10)

In [30]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input],name="input")
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [31]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow [supported by tfllte]
    lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0) 

    # Get lstm cell output [supported by tfllte]
    outputs, states = tf.compat.v1.nn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [32]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits, name="model_output")

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [33]:
saver = tf.train.Saver()

In [34]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    num_of_batches =  int(len(input_train)/batch_size)
    

    for step in range(1, training_steps+1):
        
        ptr = 0
        
        for j in range(num_of_batches):
            
            batch_x, batch_y = input_train[ptr:ptr+batch_size], output_train[ptr:ptr+batch_size]

            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            
            ptr = ptr + batch_size
            
            if (step % display_step == 0) or step == 1:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: input_train,
                                                                     Y: output_train})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

    print("Optimization Finished!")
    
    saver.save(sess, "lstm_model_v1/lstm-model.ckpt")
    
    print("model saved")
    
    tf.train.write_graph(sess.graph.as_graph_def(), '.', 'lstm_model_v1.pbtxt', as_text=True)
    
    print("graph saved")


Step 1, Minibatch Loss= 5.1239, Training Accuracy= 0.200
Step 200, Minibatch Loss= 1.1123, Training Accuracy= 0.800
Step 400, Minibatch Loss= 0.8893, Training Accuracy= 0.800
Step 600, Minibatch Loss= 0.7732, Training Accuracy= 1.000
Step 800, Minibatch Loss= 0.6951, Training Accuracy= 1.000
Step 1000, Minibatch Loss= 0.6360, Training Accuracy= 1.000
Optimization Finished!
model saved
graph saved


In [35]:
### freezing graph

from tensorflow.python.tools import freeze_graph

x = freeze_graph.freeze_graph('lstm_model_v1.pbtxt', "", False, 
                          'lstm_model_v1/lstm-model.ckpt', "model_output",
                           "save/restore_all", "save/Const:0",
                           'frozen_lstm_model_v1.pb', True, ""  
                         )

INFO:tensorflow:Restoring parameters from lstm_model_v1/lstm-model.ckpt
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


In [36]:
### covert to tflite

graph_def_file = "frozen_lstm_model_v1.pb"
input_arrays = ["input"]
output_arrays = ["model_output"]

converter = tf.lite.TFLiteConverter.from_frozen_graph(
        graph_def_file, input_arrays, output_arrays)
tflite_model = converter.convert()
open("converted_lstm_model_v1.tflite", "wb").write(tflite_model)

284856

In [37]:
### doing inference of tflite model

import numpy as np

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_lstm_model_v1.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


input_data = np.array([[[1],[2],[3],[4],[5]]], dtype=np.float32)

print("input tensor:")
print(input_data)

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.

output_data = interpreter.get_tensor(output_details[0]['index'])
print("output tensor:")
print(output_data)

print("\nprediction : "+str(np.argmax(output_data)) )

input tensor:
[[[1.]
  [2.]
  [3.]
  [4.]
  [5.]]]
output tensor:
[[1.01298548e-03 4.11978195e-04 5.37649903e-04 1.42181350e-03
  1.97638854e-01 5.55840496e-04 1.12530324e-04 5.94648004e-01
  1.70900747e-01 2.95993574e-02 3.16017610e-03]]

prediction : 7


In [38]:
# Optimize for inference (removes params used in training not needed for inference)

from tensorflow.python.tools import optimize_for_inference_lib

input_graph_def = tf.GraphDef()
with tf.gfile.Open("frozen_lstm_model_v1.pb", "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def,
        ["input"], # an array of the input node(s)
        ["model_output"], # an array of output nodes
        tf.float32.as_datatype_enum)


# Save the optimized graph

f = tf.gfile.FastGFile("optimal_lstm_model_v1.pb", "w")
f.write(output_graph_def.SerializeToString())